In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import warnings 
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

# 통신데이터 합치기 및 카테고리별로 데이터프레임 나누기

In [38]:
for m in tqdm(range(1,10),desc='월단위 진행'):
    globals()["ts{}".format(m)] = pd.read_csv(f'./data/2022.{m}월_29개 통신정보.csv',encoding='cp949')
    try : 
        globals()["ts{}".format(m)].drop(columns=['행정동코드','총인구수','1인가구수'],inplace=True)    
    except:
        globals()["ts{}".format(m)].drop(columns=['행정동코드','총인구','1인가구수'],inplace=True)  
    # globals()["ts{}".format(m)].drop(columns=globals()["ts{}".format(m)].filter(like='4분위수'),inplace=True) 
    # 데이터가 평균값을 제시하고 그 외 4분위 관련 수치를 따로 제공해주고 있음 -> 이미 전처리가 되어있는 거 같음..?
    globals()["ts{}".format(m)]['연령대_1'] = globals()["ts{}".format(m)]['연령대']//10*10
    globals()["ts{}".format(m)]['월'] = int(m)
    fix = globals()["ts{}".format(m)][['월','연령대_1','자치구','성별']]
    uni = ['상주지','출근','근무','소액결재','연체','카카오톡','SNS','통화','문자','데이터','이동','체류','지하철','게임','금융','쇼핑','동영상','유튜브','넷플릭스','배달']

    for col in tqdm(uni,desc=f'{m}월'):
        globals()["{}_{}".format(col,m)] = pd.concat([fix,globals()["ts{}".format(m)].filter(like=col)],axis=1)

for col in tqdm(uni,desc='카테고리별로 합치기'):
    globals()["{}".format(col)] = pd.DataFrame()
    for m in range(1,10):
        globals()["{}".format(col)]= pd.concat([globals()["{}".format(col)],globals()["{}_{}".format(col,m)]])


월단위 진행:   0%|          | 0/9 [00:00<?, ?it/s]

1월:   0%|          | 0/20 [00:00<?, ?it/s]

2월:   0%|          | 0/20 [00:00<?, ?it/s]

3월:   0%|          | 0/20 [00:00<?, ?it/s]

4월:   0%|          | 0/20 [00:00<?, ?it/s]

...

6월:   0%|          | 0/20 [00:00<?, ?it/s]

7월:   0%|          | 0/20 [00:00<?, ?it/s]

8월:   0%|          | 0/20 [00:00<?, ?it/s]

9월:   0%|          | 0/20 [00:00<?, ?it/s]

카테고리별로 합치기:   0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
len(uni)

20

In [39]:
def lst_chunk(lst,n):
    return [lst[i:i+n] for i in range(0,len(lst),n)]

uni_dv = lst_chunk(uni,5)
uni_dv

[['상주지', '출근', '근무', '소액결재', '연체'],
 ['카카오톡', 'SNS', '통화', '문자', '데이터'],
 ['이동', '체류', '지하철', '게임', '금융'],
 ['쇼핑', '동영상', '유튜브', '넷플릭스', '배달']]

In [25]:
게임_8.shape[0], ts8.shape[0]

(10176, 10176)

In [43]:
게임.describe()

,월,연령대_1,성별,게임 서비스 사용 미추정 인구수,게임 서비스 사용일수,"게임 서비스 사용일수: 4분위수, 25%","게임 서비스 사용일수: 4분위수, 50%","게임 서비스 사용일수: 4분위수, 75%"
count,91584.000000,91584.000000,91584.000000,91584.000000,91584.000000,91584.000000,91584.000000,91584.000000
mean,5.000000,45.000000,1.500000,38.540935,45.815114,27.190345,38.436058,58.063953
std,2.582003,17.078345,0.500003,38.883687,13.939530,5.945774,10.658400,25.049529
min,1.000000,20.000000,1.000000,0.000000,20.400000,3.300000,18.300000,23.300000
25%,3.000000,30.000000,1.000000,17.000000,35.700000,23.300000,31.700000,40.000000
50%,5.000000,45.000000,1.500000,30.000000,41.400000,26.700000,33.300000,48.300000
75%,7.000000,60.000000,2.000000,46.000000,55.400000,31.700000,43.300000,70.000000
max,9.000000,70.000000,2.000000,542.000000,109.800000,73.300000,131.700000,150.000000


# 카테고리별로 분류 모델링 돌리고 평가지표 확인(전처리 아무것도 안한 데이터)

In [ ]:
uni_df = [상주지,출근,근무,소액결재,연체,카카오톡,SNS,통화,문자,데이터,이동,체류,지하철,게임,금융,쇼핑,동영상,유튜브,넷플릭스,배달]
uni = ['상주지','출근','근무','소액결재','연체','카카오톡','SNS','통화','문자','데이터','이동','체류','지하철','게임','금융','쇼핑','동영상','유튜브','넷플릭스','배달']

for name_df,name in zip(tqdm(uni_df),uni):
    test = name_df.drop(columns=['자치구','월'])
    y = test[['연령대_1']]
    X=test.drop(columns='연령대_1')
    
    #연령 인코딩 해주기
    le = LabelEncoder()
    result = le.fit_transform(y['연령대_1'])
    # print(le.classes_)
    
    # 성별 인코딩 해주기 
    le = LabelEncoder()
    X['성별'] = le.fit_transform(X['성별'])
    # print(le.classes_)

    X_train,X_test, y_train,y_test = train_test_split(X,result,test_size=0.2,random_state=11)
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    pred_dt = dt.predict(X_test)
    pred_proba_dt = dt.predict_proba(X_test)
    
    rf = RandomForestClassifier()
    rf.fit(X_train,y_train)
    pred_rf = rf.predict(X_test)
    pred_proba_rf = rf.predict_proba(X_test)
    
    print(f'=== [{name}] 정보 분류 모형 성능 평가 ===')
    print(f'DT accuracy : {accuracy_score(y_test,pred_dt)} \nDT f1(macro) : {f1_score(y_test,pred_dt,average="macro")} \nDT logloss : {log_loss(y_test,pred_proba_dt)}\n')
    print(f'RF accuracy : {accuracy_score(y_test,pred_rf)} \nRF f1(macro) : {f1_score(y_test,pred_rf,average="macro")} \nRF logloss : {log_loss(y_test,pred_proba_rf)}\n')
    
    # print(classification_report(y_test,pred))